In [ ]:
import pandas as pd
import config 

hny_pap_nl = pd.read_csv("data/hny/pap-nl.csv", index_col=0)
stparkpap_pap_nl_name = "-".join(config.scrapeTarget["languages"])
stparkpap_pap_nl = pd.read_csv(f"data/stparkpap/{stparkpap_pap_nl_name}.csv")#, index_col=0)
df = hny_pap_nl
df.index.name
stparkpap_pap_nl

In [ ]:
from translate import Translate
trans = Translate()

short_sentences = [
    "ami ta bai kas",
    "mi stima mi yu-homber",
    "ami ta kòre outo",
    "mi no kier bai skol",
    "mi no kier kumpra bo kacho",
    "e ta hopi grandi",
    ]
long_sentences = [
    # "mi no ta guste pasobra e ta parse un kabritu",
    # "nan no tabata tin ningun idea pakiko e no a hasie"
    # "mi kier duna bo un kos aki",
    # "Mi no kier tende loke bo ta bisa",
    # "Mi no kier tende loke bo ta bisa, pasobra tur biaha bo ta gaña",
    # "Mi kier kumpra un grandi i un chiki",
    # "Mi tin hopi gana di bai landa i kome na mi mama su kas"
    "Mi no kier kita mi karson",
    "antiano no sa biba den sushi asinaki mes"
]

for sentence in long_sentences:
    translation = ""
    for word in sentence.split():
        word = word.lower()
        Translate.attachClosest(df, word)

        translation += df.iloc[0]["nl-simple"] + " "
    corrected = trans.correctSentence(translation)
    print(f"source: {sentence} \ntranslation: {translation}\ncorrected: {corrected}")
    # print(f"source: {sentence} \ndest: {corrected}")

In [ ]:
import difflib


# difflib.get_close_matches(test_sentence, src_sent, n = 2, cutoff=0.8)
# score = difflib.SequenceMatcher(None, test_sentence, src_sent[1]).ratio()
def getClosestSentences(sentence, lookup_sentences, lan):
    all_sentences = lookup_sentences.copy()
    all_sentences["closest"] = lookup_sentences.apply(lambda row: min( 
        difflib.SequenceMatcher(None, Translate.remove_accents(row[lan]), sentence).ratio() + 0.001,
        difflib.SequenceMatcher(None, row[lan], sentence).ratio()
    ), axis=1)
    all_sentences.sort_values(by="closest", inplace= True, ascending=False)
    return all_sentences.head(3)

sentences = stparkpap_pap_nl[stparkpap_pap_nl["type"] == "sentence"][["pap-simple","nl-simple"]]

In [ ]:

# Sliding window?
# Replace sub sentences
# Longer ones get priority
# In future make smarter instead of greedy to match most words
def getSubSentences(sentence):
    words = sentence.split(" ")
    subsentences = []
    for i in range(2, len(words) + 1):
        
        for j in range(0, len(words) - i + 1):
            cur_sentence = words[j:j+i]
            subsentences.append(cur_sentence)
    # print(subsentences)
    return subsentences

subsentences = getSubSentences(test)
sentence_translations = {}
for sub in subsentences[:5]:
    sentence = " ".join(sub)
    closest = getClosestSentences(sentence, sentences, "pap-simple")
    if closest.iloc[0,-1] > 0.9:
        print("Found a close one")
        print(f"sub:{sub},closest:{closest}")